## **Селекция признаков (кодирование категориальных признаков и их полезность)**

Для этого упражнения я выбрала датасет "Анализ портрета клиента". 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import seaborn as sns
from sklearn import preprocessing


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/marketing_campaign.csv', sep = '\t')
df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2216 entries, 0 to 2239
Data columns (total 29 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   2216 non-null   int64  
 1   Year_Birth           2216 non-null   int64  
 2   Education            2216 non-null   object 
 3   Marital_Status       2216 non-null   object 
 4   Income               2216 non-null   float64
 5   Kidhome              2216 non-null   int64  
 6   Teenhome             2216 non-null   int64  
 7   Dt_Customer          2216 non-null   object 
 8   Recency              2216 non-null   int64  
 9   MntWines             2216 non-null   int64  
 10  MntFruits            2216 non-null   int64  
 11  MntMeatProducts      2216 non-null   int64  
 12  MntFishProducts      2216 non-null   int64  
 13  MntSweetProducts     2216 non-null   int64  
 14  MntGoldProds         2216 non-null   int64  
 15  NumDealsPurchases    2216 non-null   i

In [ ]:
#все имеющиеся значения столбца Education
#df['Education'].value_counts()

In [ ]:
#Превращение категориальных признаков в числовые
coder = preprocessing.LabelEncoder()      #создали кодер
for name in ['Education', 'Marital_Status', 'Dt_Customer']: #все имеющиеся категориальные признаки
  coder.fit(df[name])                     #подаем сюда столбец, который хотим преобразовать
  df[name] = coder.transform(df[name])    #а здесь уже преобразуем этот столбец в числа и перезаписываем поверх оригинальных значений

В этот датасете было всего три категориальных признака (Dtype = object): 
*    ***Marital_Status*** - семейное положение
*    ***Dt_Customer*** - дата регистрации клиента в компании
*    ***Education*** - уровень образования

Теперь они преобразованы в числовые и готовы участвовать в обучении модели.

За **целевой признак** беру уровень образования - **Education.**

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2240 entries, 0 to 2239
Data columns (total 29 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   2240 non-null   int64  
 1   Year_Birth           2240 non-null   int64  
 2   Education            2240 non-null   object 
 3   Marital_Status       2240 non-null   object 
 4   Income               2216 non-null   float64
 5   Kidhome              2240 non-null   int64  
 6   Teenhome             2240 non-null   int64  
 7   Dt_Customer          2240 non-null   object 
 8   Recency              2240 non-null   int64  
 9   MntWines             2240 non-null   int64  
 10  MntFruits            2240 non-null   int64  
 11  MntMeatProducts      2240 non-null   int64  
 12  MntFishProducts      2240 non-null   int64  
 13  MntSweetProducts     2240 non-null   int64  
 14  MntGoldProds         2240 non-null   int64  
 15  NumDealsPurchases    2240 non-null   i

### **Узнаем важность каждого признака в задаче предсказания уровня образования клиента**

Селекция признаков становится особенно актуальной, когда у нас очень много признаков и нужно определить самые ценные для нашей модели предсказания.

Значимость признака измеряется не в вакууме, а относительно того, как он предсказывает уровень образования клиента.

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
selector = ExtraTreesClassifier()    #создаю селектор
result = selector.fit(df[df.columns], df['Education'])    #а потом подаю в него все столбцы. Через запятую указывается целевой признак.
result.feature_importances_  #а эта команда выведет на экран значимость каждого из признаков в некоторой условной шкале

array([0.01202764, 0.01733582, 0.70863964, 0.01218938, 0.01621868,
       0.00675729, 0.00770418, 0.01459457, 0.0134093 , 0.02262071,
       0.01785865, 0.01465575, 0.01750883, 0.01733111, 0.01854083,
       0.01108122, 0.01243588, 0.0115755 , 0.01252594, 0.01333309,
       0.00420274, 0.00350328, 0.00288488, 0.00321127, 0.00124002,
       0.0010978 , 0.        , 0.        , 0.005516  ])

Понятно, что в таком виде очень сложно представлять информацию. Поэтому представим эти данные в понятном табличном виде:

In [ ]:
features_table = pd.DataFrame(result.feature_importances_, #создаю датафрейм и закидываю в него полученные числа
                              index = df.columns,          #в качестве индексов назначаю названия признаков
                              columns = ['importance'])    #задаю имя единственному столбцу нового датафрейма 

In [ ]:
features_table.sort_values(by = 'importance', ascending = False)   #для наглядности отсортирую строки по убыванию

,importance
Education,0.708640
MntWines,0.022621
MntGoldProds,0.018541
MntFruits,0.017859
MntFishProducts,0.017509
Year_Birth,0.017336
MntSweetProducts,0.017331
Income,0.016219
MntMeatProducts,0.014656
Dt_Customer,0.014595


А дальше нужно оставить лишь несколько признаков из топа по значимости и по ним тренировать модель. Допустим, я хочу оставить 9 признаков:
*  MntWines	0.022621
*  MntGoldProds	0.018541
*  MntFruits	0.017859
*  MntFishProducts	0.017509
*  Year_Birth	0.017336
*  MntSweetProducts	0.017331
*  Income	0.016219
*  MntMeatProducts	0.014656
*  Dt_Customer	0.014595

И дальше можно переходить к выбору и обучению ML-модели.